<a href="https://colab.research.google.com/github/ishandahal/Sequence-Data-pytorch-/blob/main/Convolution_for_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uisng Convolution (1D) with sequence data. In this case sentiment analysis.

Preparing data

In [1]:
import torch
from torchtext import data
from torchtext import datasets
import random 
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy', batch_first=True)
LABEL = data.LabelField(dtype=torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

aclImdb_v1.tar.gz:   0%|          | 98.3k/84.1M [00:00<01:29, 943kB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 58.1MB/s]


Building vocab and loading the pre-trained word embeddings 

In [2]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data,
                 max_size=MAX_VOCAB_SIZE,
                 vectors='glove.6B.100d',
                 unk_init=torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                           
 99%|█████████▉| 397837/400000 [00:16<00:00, 22253.17it/s]

Creating the iterators

In [3]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, text_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device=device)

Implementing

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):
        super(CNN, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        self.conv_0 = nn.Conv2d(in_channels=1,
                                out_channels=n_filters,
                                kernel_size=(filter_sizes[0], embedding_dim))
        
        self.conv_1 = nn.Conv2d(in_channels=1,
                                out_channels=n_filters,
                                kernel_size=(filter_sizes[1], embedding_dim))
        
        self.conv_2 = nn.Conv2d(in_channels=1,
                                out_channels=n_filters,
                                kernel_size=(filter_sizes[2], embedding_dim))
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        # text : [batch size, sent len]

        embedded = self.embedding(text)

        # embedded : [batch size, sent len, emb dim]

        embedded = embedded.unsqueeze(1) # adding channel dimension

        # embedded [batch size, 1, sent len, emb dim]

        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))

        # conved_n : [batch size, n_filters, sent len - filter_sizes[n] + 1]

        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)

        # pooled_n : [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled_0, pooled_1, pooled_2), dim=1)

        # cat : [batch size, n_filters * len(filter_sizes)]

        return self.fc(cat)

Making the model more generic

In [5]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):
        super(CNN, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels=1,
                                              out_channels=n_filters,
                                              kernel_size=(fs, embedding_dim))
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        # text : [batch size, sent len]

        embedded = self.embedding(text)

        # embedded : [batch size, sent len, emb dim]

        embedded = embedded.unsqueeze(1)

        # embedded : [batch size, 1, sent len, emb dim]

        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]

        # conved_n : [batch size, n_filters, sent len - filter_sizes[n] + 1]

        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        # pooled_n : [ batch size, n_filters ]

        cat = self.dropout(torch.cat(pooled, dim=1))

        # cat :  [batch size, n_filters * len(filter_sizes)]

        return self.fc(cat)

We can use 1d convolution as well

In [6]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [7]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

Checking the number of parameters 

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad == True)

print(f"Model has {count_parameters(model):,} trainable parameters")

Model has 2,620,801 trainable parameters


Loading pretrained embeddings 

In [9]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.4127,  0.0867, -0.2232,  ..., -0.2191,  0.0485,  1.2073],
        [-0.3148,  0.1176,  0.5727,  ..., -0.1893,  0.2597, -0.3915],
        [-0.4702,  0.0093, -0.3228,  ...,  0.2960,  0.6345, -0.0689]])

Zeroing the initial weights of unknown and padding tokens

In [10]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

### Train the Model

In [11]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

Function for accuracy

In [12]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e, if you get 8/10 right, this return 0.8 not 8
    """

    # round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() # convert to float for division 
    acc = correct.sum() / len(correct)
    return acc

Defining a function to train

In [13]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()

        predictions = model(batch.text).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Defining a function for evaluation

In [14]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            predictions = model(batch.text).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Keeing track of time 

In [15]:
import time 

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Train the model 

In [16]:
n_epochs = 5

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = best_valid_loss
        torch.save(model.state_dict(), 'tut-model.pt')

    print(f"Epoch: {epoch+1:02}  |  Epoch Time: {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain Loss: {train_loss:.3f}  |  Train Acc: {train_acc*100:.2f}%")
    print(f"\tValid Loss: {valid_loss:.3f}  |  Train Acc: {valid_acc*100:.2f}%")

 99%|█████████▉| 397837/400000 [00:30<00:00, 22253.17it/s]

Epoch: 01  |  Epoch Time: 0m 19s
	Train Loss: 0.656  |  Train Acc: 61.23%
	Valid Loss: 0.514  |  Train Acc: 77.74%
Epoch: 02  |  Epoch Time: 0m 19s
	Train Loss: 0.428  |  Train Acc: 80.52%
	Valid Loss: 0.350  |  Train Acc: 84.76%
Epoch: 03  |  Epoch Time: 0m 19s
	Train Loss: 0.298  |  Train Acc: 87.61%
	Valid Loss: 0.312  |  Train Acc: 86.95%
Epoch: 04  |  Epoch Time: 0m 19s
	Train Loss: 0.219  |  Train Acc: 91.25%
	Valid Loss: 0.331  |  Train Acc: 86.35%
Epoch: 05  |  Epoch Time: 0m 19s
	Train Loss: 0.154  |  Train Acc: 94.36%
	Valid Loss: 0.315  |  Train Acc: 87.47%


Evaluating the model on the test set

In [19]:
model.load_state_dict(torch.load('tut-model.pt'))

test_loss, test_acc = evaluate(model, text_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.366 | Test Acc: 85.06%


User input 

In [25]:
import spacy

nlp = spacy.load('en')

def predict_sentiment(model, sentence, min_len=5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [27]:
predict_sentiment(model, 'I love this movie so mcuh')

0.7580788731575012

In [38]:
predict_sentiment(model, 'This is a disgusting movie')

0.2986440062522888

In [36]:
predict_sentiment(model, 'I am a big fan of the actor in the movie.')

0.5235791206359863